***GENERATED CODE FOR realestateappraisalvalueprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run realestateappraisalvaluepredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	cleaneddataetnew = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-da23aec3-d400-43ab-aa9f-3f768005d621-c000.csv', 'filename': '1707703713Cleaned datasetnew.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Real Estate Sales 730 Days/Cleaned datasetnew.csv', 'viewFileName': 'Cleaned datasetnew.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run realestateappraisalvaluepredictionHooks.ipynb
try:
	#transformationPreExecutionHook()

	realestateappraisalvaluepredictionautofe = TransformationMain.run(cleaneddataetnew,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "PropertyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "6463.71", "stddev": "5799.93", "min": "0", "max": "29427", "missing": "0"}, "updatedLabel": "PropertyID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrCompositeLandUseID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "9.55", "stddev": "47.02", "min": "0", "max": "455", "missing": "0"}, "updatedLabel": "xrCompositeLandUseID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrBuildingTypeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "3.98", "stddev": "3.49", "min": "0", "max": "30", "missing": "0"}, "updatedLabel": "xrBuildingTypeID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LocationStartNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "112.58", "stddev": "205.13", "min": "0", "max": "2708", "missing": "0"}, "updatedLabel": "LocationStartNumber"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrPrimaryNeighborhoodID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "109.02", "stddev": "105.14", "min": "0", "max": "492", "missing": "0"}, "updatedLabel": "xrPrimaryNeighborhoodID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LandSF", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "6687.98", "stddev": "4506.3", "min": "0", "max": "89930", "missing": "0"}, "updatedLabel": "LandSF"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalFinishedArea", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "1914.52", "stddev": "5054.11", "min": "0.0", "max": "293440.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalFinishedArea"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LivingUnits", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "1.15", "stddev": "4.36", "min": "0", "max": "252", "missing": "0"}, "updatedLabel": "LivingUnits"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrSalesValidityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "9.14", "stddev": "10.74", "min": "0", "max": "27", "missing": "0"}, "updatedLabel": "xrSalesValidityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrDeedID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3679", "mean": "2.98", "stddev": "3.71", "min": "0", "max": "9", "missing": "0"}, "updatedLabel": "xrDeedID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ParcelID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "1210.55", "stddev": "920.02", "min": "0.0", "max": "3026.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ParcelID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ApartmentUnitNumber_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "32.64", "stddev": "79.86", "min": "0.0", "max": "425.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ApartmentUnitNumber_strin..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StreetNameAndWay_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "92.05", "stddev": "93.19", "min": "0.0", "max": "394.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StreetNameAndWay_stringin..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OwnerLastName_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "510.98", "stddev": "547.93", "min": "0.0", "max": "1858.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "OwnerLastName_stringindex..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OwnerFirstName_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "265.61", "stddev": "373.02", "min": "0.0", "max": "1334.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "OwnerFirstName_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryGrantor_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "1084.53", "stddev": "958.5", "min": "0.0", "max": "2951.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryGrantor_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SaleDate_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "75.98", "stddev": "71.75", "min": "0.0", "max": "313.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SaleDate_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SalePrice _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "51.23", "stddev": "98.27", "min": "0.0", "max": "500.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SalePrice _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalAppraisedValue _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "345.03", "stddev": "332.21", "min": "0.0", "max": "1259.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalAppraisedValue _stri..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LegalReference_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "1307.85", "stddev": "1055.46", "min": "0.0", "max": "3263.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LegalReference_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AssrLandUse_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3679", "mean": "1.39", "stddev": "1.24", "min": "0.0", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AssrLandUse_stringindexer"}]}))

	#transformationPostExecutionHook(realestateappraisalvaluepredictionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run realestateappraisalvaluepredictionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(realestateappraisalvaluepredictionautofe, ["PropertyID", "xrCompositeLandUseID", "xrBuildingTypeID", "LocationStartNumber", "xrPrimaryNeighborhoodID", "LandSF", "TotalFinishedArea", "LivingUnits", "xrSalesValidityID", "xrDeedID", "ParcelID_stringindexer", "ApartmentUnitNumber_stringindexer", "StreetNameAndWay_stringindexer", "OwnerLastName_stringindexer", "OwnerFirstName_stringindexer", "PrimaryGrantor_stringindexer", "SaleDate_stringindexer", "SalePrice _stringindexer", "LegalReference_stringindexer", "AssrLandUse_stringindexer"], "TotalAppraisedValue _stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

